In [61]:
import pandas as pd
from ast import literal_eval
import re

In [62]:
df = pd.read_csv("../Data/Raw/movies_metadata.csv")
df.head(1)

C:\Users\MD Maruf\AppData\Local\Temp\ipykernel_30628\1899124903.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/Raw/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [63]:
df.shape

(45466, 24)

In [64]:
df = df.drop(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id', 'popularity', 'tagline', 'title', 'poster_path', 'production_companies', 'production_countries', 'original_language', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'video'], axis=1)
df.head()

,genres,original_title,overview,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,5.7,173.0


In [65]:
df['overview'].isna().sum()

np.int64(954)

In [66]:
df['overview'] = df['overview'].fillna(" ")

In [67]:
print(f"Vote Count NAN value: {df['vote_count'].isna().sum()}")
print(f"Vote Average NAN value: {df['vote_average'].isna().sum()}")

Vote Count NAN value: 6
Vote Average NAN value: 6


In [68]:
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

df = df.dropna(subset=['vote_count'])
df = df.dropna(subset=['vote_average'])

In [69]:
df.shape

(45460, 5)

In [70]:
df.loc[2345]['genres']

"[{'id': 28, 'name': 'Action'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}]"

In [71]:
df['genres'] = df['genres'].apply(literal_eval)
df['genres'] = df['genres'].apply(lambda x: " ".join([genre['name'] for genre in x]))

In [72]:
df[['original_title', 'genres']].head()

,original_title,genres
0,Toy Story,Animation Comedy Family
1,Jumanji,Adventure Fantasy Family
2,Grumpier Old Men,Romance Comedy
3,Waiting to Exhale,Comedy Drama Romance
4,Father of the Bride Part II,Comedy


In [73]:
df[['overview', 'genres']].isna().sum()

overview    0
genres      0
dtype: int64

In [74]:
df['description'] = df['overview'] + " " + df['genres']

In [75]:
df.loc[6754]['description']

'Composer Gordon Stuart brutaly rapes a fashion model, goes to trial, gets freed, comes back and rapes her little sister. She takes revenge. Drama Thriller'

In [76]:
df.loc[9875]['description']

'Ming dynasty noblewoman Yang must escape from the evil eunuch Hsu. She seeks refuge at a decrepit town where she gets assistance from a naive scholar & a group of mysterious yet powerful monks. Action Adventure'

In [77]:
def clean_text_column(text):
    text = text.str.lower()
    text = text.str.replace(r"\\'", "'", regex=True)
    text = text.apply(lambda x: re.sub(r"[^\w\s']", " ", x))
    text = text.str.strip()
    text = text.str.replace(r"\s+", " ", regex=True)
    
    return text

df['description'] = clean_text_column(df['description'])

df[['original_title', 'description']].head()

,original_title,description
0,Toy Story,led by woody andy's toys live happily in his r...
1,Jumanji,when siblings judy and peter discover an encha...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,cheated on mistreated and stepped on the women...
4,Father of the Bride Part II,just when george banks has recovered from his ...
